In [14]:
class field(object):
    def __init__(self, massDim, LORindices, ONindices, symbol):
        self.massDim = massDim #int
        self.LORindices = LORindices #list of strings
        self.ONindices = ONindices #list of strings
        self.LORrank = len(LORindices)
        self.ONrank = len(ONindices)
        self.symbol = symbol #string representing field
        self.tensorRep = '' #Create string showing tensor structure, without showing decomposition in terms of
                            #elementary fields and derivatives
        self.elementary = [] #Stores list of N elementary field factors phi and dphi
        self.contractions = [] #Stores the indices with respect to which successive elementary factors are linked
                                #e.g., [[0,1],[1,0],[0,1]] for phi^i1 dphi_u1^i1 dphi_u1^i2 phi^i2, [[1,1]] for dphi_u^i dphi_u^i. For N 
                                #elementary fields, N-1 links.
        self.decomposition = '' #String showing structure of contractions in terms of elementary fields and 
                                #derivatives 
    def __str__(self): #Prints out attributes of field object
        return 'massDim: ' + str(self.massDim) + '\n' \
            + 'LORindices: ' + str(self.LORindices) + '\n' \
            + 'ONindices: ' + str(self.ONindices) + '\n' \
            + 'symbol: ' + str(self.symbol) + '\n' \
            + 'LORrank: ' + str(self.LORrank) + '\n' \
            + 'ONrank: ' + str(self.ONrank) + '\n' \
            + 'tensorRep: ' + str(self.tensorRep)
    
    #Define get and set functions
    def get_massDim(self):
        return self.massDim
    def get_LORindices(self):
        return self.LORindices
    def get_ONindices(self):
        return self.ONindices
    def get_LORrank(self):
        return self.LORrank
    def get_ONrank(self):
        return self.ONrank
    def get_symbol(self):
        return self.symbol
    def get_tensorRep(self):
        return self.tensorRep
    def get_elementary(self):
        return self.elementary
    def get_contractions(self):
        return self.contractions
    def get_tensorRep(self):
        return self.tensorRep
    
    def set_massDim(self, massDim):
        self.massDim = massDim
    def set_LORindices(self, LORindices):
        self.LORindices = LORindices
    def set_ONindices(self, ONindices):
        self.ONindices = ONindices
    def set_symbol(self, symbol):
        self.symbol = symbol
    def set_tensorRep(self, tensorRep):
        self.tensorRep = tensorRep
    def set_elementary(self, elementary):
        self.elementary = elementary
    def set_contractions(self, contractions):
        self.contractions = contractions
    def set_tensorRep(self):
        tensor = self.symbol
        for i in self.LORindices:
            tensor = tensor + '_' + str(i)
        for j in self.ONindices:
            tensor = tensor + '^' + str(j)
        self.tensorRep = tensor
        return tensor
    #No set_LORrank() or set_ONrank() functions since LORrank and ONrank are determined by length of index lists
        
    #Returns a new field corresponding to derivative of field by increasing massDim and LORindex by one
    def derivative(self, mu = 'u'): 
        massDim_new = self.get_massDim() + 1
        #print(self.get_LORindices())
        LORindices_new = self.get_LORindices() + [mu]
        #print(LORindices_new)
        #LORrank_new = self.get_LORrank() + 1
        fieldDeriv = field(massDim_new, LORindices_new, self.ONindices, 'D' + self.symbol)
        return fieldDeriv 
    
    #Return string showing structure in terms of elementary fields and derivatives
    def decompose(self):
        '''
        OUTPUT
        Returns string showing decomposition in terms of elementary fields and derivatives
        '''
        
        elementary = self.get_elementary()[:]
        contractions = self.get_contractions()[:]
        decomposition = ''
        
        LOR_count = 0 #counter for numbering contracted LOR indices
        ON_count = 0 #counter for numbering contracted ON indices
        
        for i in range(len(contractions)):
            
            e_current = elementary[i]
            e_next = elementary[i+1]
            
            e_current.set_tensorRep()
            e_next.set_tensorRep()
            
            dot = contractions[i]
            
            if dot[0]: #Check if Lorentz index is contracted
                
                LORindex = 'U' + str(LOR_count)
                
                e_current.set_LORindices([LORindex])
                e_current.set_tensorRep()
                
                e_next.set_LORindices([LORindex])
                e_next.set_tensorRep()
                
                LOR_count += 1
            
            else:
                pass
            
            if dot[1]: #Check if ON index is contracted
                
                ONindex = 'I' + str(ON_count)
                
                e_current.set_ONindices([ONindex])
                e_current.set_tensorRep()
                
                e_next.set_ONindices([ONindex])
                e_next.set_tensorRep()
            
                ON_count += 1
            
            else:
                pass
            
            
            if i < (len(contractions)-1):
                e_current_string = e_current.get_tensorRep()
                e_next_string = e_next.get_tensorRep()
                decomposition += '(' + e_current_string + ')'
            else: 
                e_current_string = e_current.get_tensorRep()
                e_next_string = e_next.get_tensorRep()
                decomposition += '(' + e_current_string + ')' + '(' + e_next_string + ')'
            
        
        return decomposition
    
    
    
    def contract(self, other, LORcontract, ONcontract, symbol_prod): #don't need to know decomposition, but can look at it 
                                                        #from this level too. 
        '''
        define new field object, including with elementary, contractions, decomposition; but should be able to
        work without being given decomposition. 
        
        OUTPUT:
        new field object
        '''
        massDim_prod = self.get_massDim() + other.get_massDim()
        LORindices_prod = [] #delete contracted LOR indices 
        ONindices_prod = [] #delete contracted ON indices\
        
        #Remove indices in LORcontract from LORindices_prod and indices in ONcontract from ONindices_prod. Or
        #rather, add all indices to LORindices_prod and ONindices_prod except those indicated by LORcontract and 
        #ONcontract
        
        #Calculate LORindices, ONindices of product. [:] makes copy rather than mere reference/pointer, so that we don't alter attributes of factor fields. 
        LORindices_self = self.get_LORindices()[:]
        LORindices_other = other.get_LORindices()[:]

        ONindices_self = self.get_ONindices()[:]
        ONindices_other = other.get_ONindices()[:]
        
        
        
        LORcontracted_self = []
        LORcontracted_other = []
        
        for pair in LORcontract:
            LORcontracted_self.append(pair[0])
            LORcontracted_other.append(pair[1])
            
        ONcontracted_self = []
        ONcontracted_other = []
        
        for pair in ONcontract:
            ONcontracted_self.append(pair[0])
            ONcontracted_other.append(pair[1])
        
        
        
        for i in range(len(LORindices_self)): 
            if i not in LORcontracted_self:
                LORindices_prod.append(LORindices_self[i])
            
        for i in range(len(LORindices_other)): 
            if i not in LORcontracted_other:
                LORindices_prod.append(LORindices_other[i])
            
        
        for i in range(len(ONindices_self)): 
            if i not in ONcontracted_self:
                ONindices_prod.append(ONindices_self[i])
        
        for i in range(len(ONindices_other)): 
            if i not in ONcontracted_other:
                ONindices_prod.append(ONindices_other[i])
  
        print("LORindices_prod: " + str(LORindices_prod))
        print("ONindices_prod: " + str(ONindices_prod))
        
        prod = field(massDim_prod, LORindices_prod, ONindices_prod, symbol_prod)
        
        
        '''
        Set elementary fields 
        '''
        
        elementary_prod = [] #list of elementary fields and their derivatives associated with product field
        contractions_prod = [] #list of contractions among elementary fields 
        
        elementary_self = self.get_elementary()
        contractions_self = self.get_contractions()
        
        elementary_other = other.get_elementary()
        contractions_other = other.get_contractions()
        
        
        elementary_prod = elementary_self + elementary_other
        contractions_prod = 
        
        
        return prod
        
       
        

In [15]:
A = field(5, ['u0', 'u1', 'u2'], ['i0', 'i1'], 'A')

In [16]:
B = field(3, ['v0', 'v1'], ['j0', 'j1'], 'B')

In [19]:
prod = field.contract(A, B, [[2,1],[1,0]], [[1,1]], 'C')

LORindices_prod: ['u0']
ONindices_prod: ['i0', 'j0']


In [20]:
print(prod)

massDim: 8
LORindices: ['u0']
ONindices: ['i0', 'j0']
symbol: C
LORrank: 1
ONrank: 2
tensorRep: 


In [21]:
prod.set_tensorRep()

'C_u0^i0^j0'

In [22]:
prod.get_tensorRep()

'C_u0^i0^j0'